# Anomaly Detection Project

### Project Scenario

Hello,

I have some questions for you that I need to be answered before the board meeting Thursday afternoon. I need to be able to speak to the following questions. I also need a single slide that I can incorporate into my existing presentation (Google Slides) that summarizes the most important points. My questions are listed below; however, if you discover anything else important that I didn’t think to ask, please include that as well.

1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?

### Other info:


• To get 100 on this project you only need to answer 5 out of the 7 questions (along with the other deliverables listed below i.e. slide, your notebook, etc).

• Submit a link to a final notebook on GitHub that asks and answers questions - document the work you do to justify findings

• Compose an email with the answers to the questions/your findings, and in the email, include the link to your notebook in GitHub and attach your slide.

• You will not present this, so be sure that the details you need your leader to convey/understand are clearly communicated in the email.

• Your slide should be like an executive summary and be in form to present.

• Continue to use best practices of acquire.py, prepare.py, etc.

• Since there is no modeling to be done for this project, there is no need to split the data into train/validate/test

• The cohort schedule is in the SQL database, and alumni.codeup.com has info as well.

In [1]:
# imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from sklearn import metrics
from env import user, host, password

In [2]:
# sets up a secure connection to the Codeup db using my login infor
def get_db_url(db, user=user, host=host, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [3]:
# assigns the curriculum_logs url to the variable name 'url' so it can be used in additional functions
url = get_db_url('curriculum_logs')

In [4]:
# creating functions to use in my notebook
def get_connection(db, user=user, host=host, password=password):
    '''
    This function uses my env file to create a connection url to access
    the Codeup database. '''
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [5]:
def curriculum_data():
    '''
    This function get the curriculum data from the codeup database
    '''

    sql_query = '''select *
    FROM logs
    JOIN cohorts on cohorts.id = logs.user_id
    '''
    
    return pd.read_sql(sql_query, get_connection('curriculum_logs'))

In [6]:
def get_curriculum_data():
    '''
    Reading curriculum data from codeup database and creates a csv file into a dataframe
    '''
    if os.path.isfile('curriculum_logs.csv'):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv('curriculum_logs.csv', index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = curriculum_data()
     
        # Cache data
        df.to_csv('curriculum_logs.csv')
        
    return df

In [7]:
df = get_curriculum_data()

In [8]:
df.head() #check_yo_head

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,1,Arches,#arches,2014-02-04,2014-04-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2,Badlands,#badlands,2014-06-04,2014-08-22,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1


In [9]:
df.isna().sum()

date              0
time              0
path              0
user_id           0
cohort_id      1334
ip                0
id                0
name              0
slack             0
start_date        0
end_date          0
created_at        0
updated_at        0
deleted_at    73739
program_id        0
dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73739 entries, 0 to 73738
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        73739 non-null  object 
 1   time        73739 non-null  object 
 2   path        73739 non-null  object 
 3   user_id     73739 non-null  int64  
 4   cohort_id   72405 non-null  float64
 5   ip          73739 non-null  object 
 6   id          73739 non-null  int64  
 7   name        73739 non-null  object 
 8   slack       73739 non-null  object 
 9   start_date  73739 non-null  object 
 10  end_date    73739 non-null  object 
 11  created_at  73739 non-null  object 
 12  updated_at  73739 non-null  object 
 13  deleted_at  0 non-null      object 
 14  program_id  73739 non-null  int64  
dtypes: float64(1), int64(3), object(11)
memory usage: 8.4+ MB
